# Methods

In [ ]:
def is_almost_conjugate(G, A, B):
    # Once in a helper function, now a method of its own.
    elem_to_class = {}
    classes = G.conjugacy_classes()
    for i, C in enumerate(classes):
        for g in C:
            elem_to_class[g] = i

    A_counts = [0] * len(classes)
    B_counts = [0] * len(classes)
    for a in A:
        A_counts[elem_to_class[a]] += 1
    for b in B:
        B_counts[elem_to_class[b]] += 1
    return A_counts == B_counts

from sage.interfaces.gap import gap

def is_conjugate(G,A,B):
  return G.gap().IsConjugate(A.gap(),B.gap())

In [ ]:
def find_almost_conjugate(G):
    sgs = G.conjugacy_classes_subgroups()
    k = len(sgs)

    i=0
    j=0
    mylist=[(0,0)]
    while(i<k):
        j=0
        while(j<k):
            if(i>=j):
                j+=1
                continue
            a = sgs[i]
            ac = a.cardinality()
            b = sgs[j]
            bc = b.cardinality()
            if(ac != bc):
                j+=1
                continue

            if (is_almost_conjugate(G,a,b)):
                mylist.append(("EXAMPLE:",{"a":i,"b":j,"quotient":G.cardinality()/ac},(a.gens(),b.gens())))
                print("SUCCESS:")
                print(a.gens())
                print(b.gens())
                print("-----")
            j+=1
        i+=1


    print(mylist)

# Group Testing

#

$$G = SL_3(\mathbb{F}_2), A = \begin{pmatrix} 1 & 0 & 0 \\ * & * & * \\ * & * & * \end{pmatrix}, B = \begin{pmatrix} 1 & * & * \\ 0 & * & * \\ 0 & * & * \end{pmatrix} $$


In [ ]:
from itertools import product

F = GF(2)
G = SL(3, F)
rows = [list(row) for row in list(product([0, 1], repeat=3))]
row_1 = [1,0,0]

A = []
B = []
for row_2 in rows:
  for row_3 in rows:
    m = Matrix(F,[row_1,row_2,row_3])
    if (m in G):
      A.append(m.transpose())
      B.append(m)
A = G.subgroup(A)
B = G.subgroup(B)

print(is_almost_conjugate(G,A,B))
print(is_conjugate(G,A,B))

In [ ]:
 # Second example, however sage does not have many features for semidirect products, including conjugacy_classes and subgroups

Z_times = Integers(8).unit_group()
Z_plus = GroupExp()(AdditiveAbelianGroup([8]))

one = Z_plus.an_element()
three = one * one * one
five = three * one * one
seven = five * one * one

def to_val(a):
    if a == Z_times[0]:  return one.value # 1
    elif a == Z_times[1]: return five.value # f1
    elif a == Z_times[2]: return three.value # f0
    else: return seven.value # f1 * f0

def twist(a_2,b_1):
  return Z_plus(to_val(a_2) + b_1.value)

G = GroupSemidirectProduct(Z_times,Z_plus,twist=twist)
def createGroup(gen, relations):
  #mainly for abstract groups, we can make groups with this that are ismoprphic to some group
  #we can create the second example using this.
  F = FreeGroup(gen)
  gen = F.gens()
  return F/relations



In [ ]:
def is_simple_lift_unlinked(G,A,s,t,m,n,g):
  #this is untested I imagine this might have errors or inefficent
    conjugate=g**(-1)*s**(m)*g
    bigger_conj= g**(-1)*s**(m)*t**(n)*g

    for i in range(1,m):
        b=(g**(-1)*s**(i)*g)
        if b not in A:
            continue
        else:
            return False
    for j in range(1,n):
        checking = g**(-1)*s**(-m)*t**(j)*s**(m)*g
        if checking not in A:
            continue
        else:
            return False
    if conjugate in A and (m*n<0):
        return False
    if bigger_conj in A:
        return True
    return False

def count_simple(G,A,s,t,m,n):
    c=0
    for g in G.list():
        if is_simple_lift_unlinked(G,A,s,t,m,n,g):
            c+=1
    return c
def distinguish_simple_lifts_unlinked(G,A,B,s,t,m,n):
    return count_simple_lifts_unlinked(G,A,s,t,m,n)== count_simple_lifts_unlinked(G,B,s,t,m,n)
def distinguishable_unlinked(G,A,B):

  upperA=G.order()/A.order()
  upperB=G.order()/B.order()



# Time Tests

In [ ]:
import time

# -------------------------------
# Tests
# -------------------------------
print("Generating groups and subsets...\n")

G1 = SymmetricGroup(6)
A1 = random_subgroup(G1)
B1 = random_subgroup(G1)

G2 = SymmetricGroup(7)
A2 = random_subgroup(G2)
B2 = random_subgroup(G2)

G3 = SymmetricGroup(8)
A3 = random_subgroup(G3)
B3 = random_subgroup(G3)

G4 = GL(3, GF(3))     # 11232 elements
A4 = random_subgroup(G4)
B4 = random_subgroup(G4)

G5 = GL(3, GF(5))     # 372000 elements (heavier test)
A5 = random_subgroup(G5)
B5 = random_subgroup(G5)

tests = [
    (G1, A1, B1, "S6"),
    (G2, A2, B2, "S7"),
    (G3, A3, B3, "S8"),
    (G4, A4, B4, "GL3_GF3"),
    (G5, A5, B5, "GL3_GF5"),
]

# -------------------------------
# Helper: random subgroup generator
# -------------------------------
def random_subgroup(G, n_gens=2):
    gens = [G.random_element() for _ in range(n_gens)]
    return G.subgroup(gens)

# -------------------------------
# Benchmark runner
# -------------------------------
def time_call(func, *args):
    t0 = time.time()
    result = func(*args)
    return result, time.time() - t0

# -------------------------------
# Benchmark running
# -------------------------------
print("=== Running Benchmarks ===\n")
for (G, A, B, name) in tests:

    print(f"\nGroup: {name}, Order = {G.order()}")

    print(A.gens())
    print(B.gens())
    r3, t3 = time_call(is_almost_conjugate, G, A, B, )
    print(f"Optimized (cached)        | result={r3!s:5s} | time={t3:8.4f}s")

print("\n=== Done ===")

Generating groups and subsets...



NameError: name 'SymmetricGroup' is not defined

In [ ]:
S = SymmetricGroup(8)
sgs = S.conjugacy_classes_subgroups()
k = len(sgs)

i=0
j=0
mylist=[(0,0)]
while(i<k):
    j=0
    while(j<k):
        if(i>=j):
            j+=1
            continue
        a = sgs[i]
        b = sgs[j]
        ac = a.cardinality()
        bc = b.cardinality()
        if(ac != bc):
            j+=1
            continue

        if (is_almost_conjugate(S,a,b)):
            mylist.append(("EXAMPLE:",{"a":i,"b":j,"quotient":S.cardinality()/ac},(a.gens(),b.gens())))
            print("SUCCESS:")
            print(a.gens())
            print(b.gens())
            print("-----")
        j+=1
    i+=1


print(mylist)

# CSV Function

In [ ]:
import csv

def find_almost_conjugate_subgroups(start_k, end_k, start_i=0, start_j=0, csv_file="almost_conjugates.csv"):
    fieldnames = ["S_k", "a", "b", "quotient", "gens_a", "gens_b"]

    # Create CSV if not present
    try:
        with open(csv_file, "x", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            print(f"Created new CSV file: {csv_file}")
    except FileExistsError:
        print(f"Appending to existing file: {csv_file}")


    try:
        for k in range(start_k, end_k + 1):
            S = SymmetricGroup(k)
            sgs = S.conjugacy_classes_subgroups()
            l = len(sgs)
            i_start = start_i if k == start_k else 0

            for i in range(i_start, l):
                j_start = start_j if (k == start_k and i == start_i) else i + 1
                for j in range(j_start, l):
                    a = sgs[i]
                    b = sgs[j]
                    if a.cardinality() != b.cardinality():
                        continue
                    if is_almost_conjugate(S, a, b):
                        quotient = S.cardinality() / a.cardinality()
                        gensa = a.gens()
                        gensb = b.gens()
                        with open(csv_file, "a", newline="") as f:
                            writer = csv.DictWriter(f, fieldnames=fieldnames)
                            writer.writerow({
                                "S_k": k,
                                "a": i,
                                "b": j,
                                "quotient": quotient,
                                "gens_a": gensa,
                                "gens_b": gensb
                            })
                        print(f"SUCCESS for SymmetricGroup({k}): ({i}, {j})")
        print("All searched")

    except KeyboardInterrupt:
        print(f"\n⏸ Interrupted — resume from (k={k}, i={i}, j={j})")
        return (k, i, j)

    except Exception as e:
        print(f"\n Error at (k={k}, i={i}, j={j}): {e}")
        return (k, i, j)

    return None

# Linked/Unlinked Methods

In [ ]:
# Unlinked Code

def is_simple_lift_unlinked(G, A, s, t, m, n, g):
    g_inv = g.inverse()

    # First Condition: g^{-1} s^j g \notin A \forall 1 \le j < |m|
    for j in range(1, abs(m)):
        if g_inv * (s**j) * g in A:
            return False

    # Second Condition:  g^{-1} s^{-m} t^k s^m g \notin A \forall 1 \le k < |n|
    s_m = s**m
    s_neg_m = s**(-m)

    for k in range(1, abs(n)):
        if g_inv * s_neg_m * (t**k) * s_m * g in A:
            return False

    # Third Condition:  If g^{-1} s^m g \in A, then m and n have same sign
    elem = g_inv * s_m * g
    if elem in A:
        if (m > 0 and n < 0) or (m < 0 and n > 0):
            return False

    # Fourth Condition: g^{-1} s^m t^n g \in A
    if g_inv * s_m * (t**n) * g not in A:
        return False

    return True

def count_simple_lifts_unlinked(G, A, s, t, m, n):
    count = 0
    for g in G.list():
        if is_simple_lift_unlinked(G, A, s, t, m, n, g):
            count += 1
    return count

def distinguish_simple_lifts_unlinked(G, A, B, s, t, m, n):
    return count_simple_lifts_unlinked(G, A, s, t, m, n) != count_simple_lifts_unlinked(G, B, s, t, m, n)

def distinguishable_unlinked(G, A, B, s,t):
    bound = G.order() // A.order()

    for m in range(-bound, bound + 1):
        if m == 0:
            continue

        for n in range(-bound, bound + 1):
            if n == 0:
                  continue

            if distinguish_simple_lifts_unlinked(G, A, B, s, t, m, n):
                return True

    return False


def is_simple_lift_linked(G, A, s, t, m, n, g):
    if g*s**(m)*t**(n)*g**(-1) not in A:
        return False
    for j in range(1,m):
        for k in range (1,n):
            if g*s**(m)*t**(k)*s**(-j)*g**(-1) in A:
                return False
    if g**(-1)*s**(m)**g in A and m*n>0:
        return True

def count_simple_lifts_linked(G, A, s, t, m, n):
  count = 0
  for g in G.list():
      if is_simple_lift_linked(G, A, s, t, m, n, g):
          count += 1
  return count

def distinguish_simple_lifts_linked(G, A, B, s, t, m, n):
  # calls count_simple_lifts_linked(G, A, s, t, m, n):
  return count_simple_lifts_linked(G, A, s, t, m, n) != count_simple_lifts_linked(G, B, s, t, m, n)

def distinguishable_linked(G,A,B,T):
  # calls distinguish_simple_lifts_linked(G, A, B, s, t, m, n):
  return "Make Method"

def distinguishable_overall(G,A,B,T):
  for s in T:
        for t in T:
            if s == t:
                continue
            distinguishable_linked(G,A,B,s,t) and distinguishable_unlinked(G,A,B,s,t)
